# WindFarmer API demo: How to use CFD.ML solely for blockage correction?
CFD.ML may be use to derive a blockage correction to a wakes-only, engineering model.
This demo demonstrated how to do it.  

In [1]:
import os
import requests
import json
import time
from copy import deepcopy

To access the API you need a authorization token. This should be kept secure - and not added to source control, so I'm getting it from an environment variable.

The token should be passed as an Authorization header. We also need to set the `Content-Type` to let the API know that we're sending JSON data.

In [2]:
api_url = 'https://windfarmer.dnv.com/api/v2/'
auth_token = os.environ['WINDFARMER_ACCESS_KEY']
headers = {
    'Authorization': f'Bearer {auth_token}',
    'Content-Type': 'application/json'
}

### Call `Status`

In [3]:
response = requests.get(api_url + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

Response from Status: 200
{"message":"Connection to DNV WindFarmer Services API was successful.","windFarmerServicesAPIVersion":"2.3.0","calculationLibraryVersion":"2.2.31.0"}


### Hypothetical wind farm in flat terrain: `The Bowl`

In [4]:
# load the inputs
with open('..\..\..\DemoData\TheBowl\TheBowl.json') as f:
    json_string = f.read()
    input_data = json.loads(json_string)

The wind farm case is presented in more detail in demo notebook: "2_long_demo-the_bowl.ipynb", skipping the layout plotting here

### Call `AnnualEnergyProduction` endpoint

Let's use EddyViscosity+LWF for wake modeling, but use CFD.ML to (bulk) correct for the impact of blockage.

In [5]:
input_data['energyEfficienciesSettings']['blockageModel']['blockageModelType'] = "CFDML" 
input_data['energyEfficienciesSettings']['blockageModel']['cfdml']['blockageCorrectionApplicationMethod'] = "OnEnergy" 
input_data['energyEfficienciesSettings']['wakeModel']['wakeModelType'] = 'EddyViscosity'


Making a call to the WFer Services API

In [6]:
start = time.time()
response = requests.post(
    api_url + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data)
print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response.status_code != 200:
    print(json.loads(response.content)['detail'])

Response 200 - OK in 251.46s


### Results

In [7]:
#extract the most important farm-level numbers
result = json.loads(response.content)
full_aep_MWh_per_year = float(result['windFarmAepOutputs'][0]['fullAnnualEnergyYield_MWh_per_year'])
cfdml_correction = float(result['weightedBlockageEfficiency'])
print(f'Wake Affected Annual Energy Production = {full_aep_MWh_per_year:.1f} MWh / year')
print(f'CFD.ML Blockage correction efficiency = {cfdml_correction:.3f} ')
print(f'Wake & Blockage Affected Annual Energy Production = {full_aep_MWh_per_year * cfdml_correction:.1f} MWh / year')

Wake Affected Annual Energy Production = 1440430.8 MWh / year
CFD.ML Blockage correction efficiency = 0.981 
Wake & Blockage Affected Annual Energy Production = 1412504.6 MWh / year
